In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import OneCycleLR
from torchsummary import summary
from tqdm import tqdm
import numpy as np

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"Using device: {device}")

class Cutout(object):
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)
        for n in range(self.n_holes):
            y, x = np.random.randint(h), np.random.randint(w)
            y1, y2 = np.clip(y - self.length // 2, 0, h), np.clip(y + self.length // 2, 0, h)
            x1, x2 = np.clip(x - self.length // 2, 0, w), np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0.
        mask = torch.from_numpy(mask).expand_as(img)
        return img * mask

torch.manual_seed(1)
batch_size = 128
train_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10, fill=0.1307),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    Cutout(n_holes=1, length=16)
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True, transform=train_transforms), batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, transform=test_transforms), batch_size=batch_size, shuffle=True, **kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(16), nn.ReLU(), nn.Dropout(0.1),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.transblock1 = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, bias=False),
            nn.BatchNorm2d(16), nn.ReLU()
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.output_conv = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.transblock1(x)
        x = self.convblock2(x)
        x = self.gap(x)
        x = self.output_conv(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

def train(model, device, train_loader, optimizer, scheduler):
    model.train()
    pbar = tqdm(train_loader)
    correct, processed = 0, 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc=f'Loss={loss.item():.4f} LR={scheduler.get_last_lr()[0]:.6f} Acc={100*correct/processed:0.2f}')

def test(model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Avg loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.2f}%)\n')

EPOCHS = 20
model = Net().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
scheduler = OneCycleLR(optimizer,
                       max_lr=0.02, 
                       steps_per_epoch=len(train_loader),
                       epochs=EPOCHS,
                       anneal_strategy='linear')

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}")
    train(model, device, train_loader, optimizer, scheduler)
    test(model, device, test_loader)

Using device: cuda


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 456kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.76MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 32, 28, 28]           4,608
       BatchNorm2d-6           [-1, 32, 28, 28]              64
              ReLU-7           [-1, 32, 28, 28]               0
           Dropout-8           [-1, 32, 28, 28]               0
         MaxPool2d-9           [-1, 32, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]             512
      BatchNorm2d-11           [-1, 16, 14, 14]              32
             ReLU-12           [-1, 16, 14, 14]               0
           Conv2d-13           [-1, 32, 14, 14]           4,608
      BatchNorm2d-14           [-1, 32,

Loss=0.6672 LR=0.004001 Acc=57.41: 100%|██████████| 469/469 [00:23<00:00, 19.66it/s]



Test set: Avg loss: 0.4810, Accuracy: 8524/10000 (85.24%)

Epoch 2


...


Test set: Avg loss: 0.0267, Accuracy: 9910/10000 (99.10%)

Epoch 19


Loss=0.3615 LR=0.001426 Acc=90.82: 100%|██████████| 469/469 [00:24<00:00, 18.96it/s]



Test set: Avg loss: 0.0222, Accuracy: 9921/10000 (99.21%)

Epoch 20


Loss=0.2882 LR=-0.000003 Acc=90.94: 100%|██████████| 469/469 [00:25<00:00, 18.15it/s]



Test set: Avg loss: 0.0216, Accuracy: 9921/10000 (99.21%)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import OneCycleLR
from torchsummary import summary
from tqdm import tqdm
import numpy as np

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"Using device: {device}")

class Cutout(object):
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)
        for n in range(self.n_holes):
            y, x = np.random.randint(h), np.random.randint(w)
            y1, y2 = np.clip(y - self.length // 2, 0, h), np.clip(y + self.length // 2, 0, h)
            x1, x2 = np.clip(x - self.length // 2, 0, w), np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0.
        mask = torch.from_numpy(mask).expand_as(img)
        return img * mask

torch.manual_seed(1)
batch_size = 128
train_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10, fill=0.1307),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    Cutout(n_holes=1, length=16)
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True, transform=train_transforms), batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, transform=test_transforms), batch_size=batch_size, shuffle=True, **kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(16), nn.ReLU(), nn.Dropout(0.1),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.transblock1 = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, bias=False),
            nn.BatchNorm2d(16), nn.ReLU()
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.output_conv = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.transblock1(x)
        x = self.convblock2(x)
        x = self.gap(x)
        x = self.output_conv(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

def train(model, device, train_loader, optimizer, scheduler):
    model.train()
    pbar = tqdm(train_loader)
    correct, processed = 0, 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc=f'Loss={loss.item():.4f} LR={scheduler.get_last_lr()[0]:.6f} Acc={100*correct/processed:0.2f}')

def test(model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Avg loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.2f}%)\n')

EPOCHS = 20
model = Net().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = OneCycleLR(optimizer,
                       max_lr=0.02,
                       steps_per_epoch=len(train_loader),
                       epochs=EPOCHS,
                       anneal_strategy='linear')

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}")
    train(model, device, train_loader, optimizer, scheduler)
    test(model, device, test_loader)

Using device: cuda


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.04MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.6MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 32, 28, 28]           4,608
       BatchNorm2d-6           [-1, 32, 28, 28]              64
              ReLU-7           [-1, 32, 28, 28]               0
           Dropout-8           [-1, 32, 28, 28]               0
         MaxPool2d-9           [-1, 32, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]             512
      BatchNorm2d-11           [-1, 16, 14, 14]              32
             ReLU-12           [-1, 16, 14, 14]               0
           Conv2d-13           [-1, 32, 14, 14]           4,608
      BatchNorm2d-14           [-1, 32,

Loss=0.6838 LR=0.004001 Acc=57.49: 100%|██████████| 469/469 [00:24<00:00, 19.42it/s]



Test set: Avg loss: 0.4262, Accuracy: 8670/10000 (86.70%)

Epoch 2


...


Test set: Avg loss: 0.0210, Accuracy: 9925/10000 (99.25%)

Epoch 19


Loss=0.3406 LR=0.001426 Acc=91.11: 100%|██████████| 469/469 [00:23<00:00, 19.95it/s]



Test set: Avg loss: 0.0188, Accuracy: 9940/10000 (99.40%)

Epoch 20


Loss=0.2850 LR=-0.000003 Acc=90.99: 100%|██████████| 469/469 [00:23<00:00, 19.91it/s]



Test set: Avg loss: 0.0190, Accuracy: 9936/10000 (99.36%)

